In [ ]:
 # Import necessary libraries
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
import tensorflow as tf

# Load CIFAR-10 dataset
(x_train, y_train), (x_val, y_val) = cifar10.load_data()

# Normalize pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0

# One-hot encode the labels
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)

# Define a data pipeline with dynamic resizing and batching
def preprocess(image, label):
    image = tf.image.resize(image, [224, 224])  # Resize to VGG16 input size
    return image, label

batch_size = 32

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(preprocess).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_ds = val_ds.map(preprocess).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

# Load the pre-trained VGG16 model without the top layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all the layers in the base model to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for our specific classification task
x = Flatten()(base_model.output)  # Flatten the output of the base model
x = Dense(128, activation='relu')(x)  # Add a dense layer with 128 neurons
x = Dropout(0.5)(x)  # Add dropout for regularization
output = Dense(num_classes, activation='softmax')(x)  # Final output layer

# Create the complete model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with frozen base layers
model.fit(train_ds, validation_data=val_ds, epochs=10)

# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[-4:]:  # Unfreeze the last 4 layers
    layer.trainable = True

# Recompile the model after unfreezing
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
model.fit(train_ds, validation_data=val_ds, epochs=5)

# Save the fine-tuned model
model.save('fine_tuned_vgg16_cifar10.h5')

print("Model training and fine-tuning complete.")


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 277s 165ms/step - accuracy: 0.1424 - loss: 2.2368 - val_accuracy: 0.2217 - val_loss: 1.9695
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 293s 158ms/step - accuracy: 0.1462 - loss: 2.1420 - val_accuracy: 0.2149 - val_loss: 1.9981
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 246s 158ms/step - accuracy: 0.1537 - loss: 2.1290 - val_accuracy: 0.2328 - val_loss: 1.9320
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 262s 158ms/step - accuracy: 0.1559 - loss: 2.1194 - val_accuracy: 0.2118 - val_loss: 1.9999
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 262s 158ms/step - accuracy: 0.1566 - loss: 2.1232 - val_accuracy: 0.2294 - val_loss: 1.9782
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 262s 158ms/step - accuracy: 0.1599 - loss: 2.1061 - val_accuracy: 0.2331 - val_loss: 1.9567
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 262s 158ms/step - accuracy: 0.1569 - loss: 2.1169 - val_accuracy: 0.2226 - val_loss: 1.9759
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 262s 158ms/step - ac

Model training and fine-tuning complete.
